<a href="https://colab.research.google.com/github/Vanya-Kir/data_analysis/blob/main/%D0%9A%D0%B8%D1%80%D0%B8%D0%BB%D0%BB%D0%BE%D0%B2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ЛР_4. Классификация (Байес)Задание
**Кириллов Иван**

В качестве данных возьмем датасет описание вина (Институт фармацевтики и пищевого анализ и технологий, Виа Бригата Салерно).

Атрибуты датасета:
1. Алкоголь
2. яблочная кислота
3. Ясень
4. Щелочность золы
5. Магний
6. Всего фенолов
7. Флавоноиды
8. Нефлаваноидные фенолы
9. Проантоцианы
10. интенсивность цвета
11. оттенок
12. OD280/OD315 разбавленных вин
13. Пролин

In [26]:
#загружаем библиотеки
from sklearn.model_selection import train_test_split
import numpy

In [27]:
#наивный байесовский классификатор 
class BayesClassifier:

    def __init__(self, intervals_num=10, useLaPlace=False):
        self.intervals_num = intervals_num
        self.useLaPlace = useLaPlace

    #метод для тренировки модели
    def fit(self, Y, X):
        self.variables_num = len(X[0])
        self.categories = numpy.unique(Y).tolist()
        self.categories_num = len(self.categories)

        self.set_discrete_intervals(X)
        X = self.discretize_data(X)

        self.PY = []
        self.PXY = []
        for cat in self.categories:
            total_counter = 0
            for y in Y:
                if y == cat:
                    total_counter += 1

            self.PY.append(total_counter / len(X))

            self.PXY.append([])
            for i in range(self.variables_num):
                self.PXY[-1].append([])
                for j in range(self.intervals_num):
                    counter = 0
                    for k in range(len(X)):
                        y = Y[k]
                        x = X[k]
                        if y == cat and x[i] == j:
                            counter += 1

                    if self.useLaPlace:
                        p = ((counter + 1) / (total_counter + self.intervals_num))
                        self.PXY[-1][-1].append(p)
                    else:
                        self.PXY[-1][-1].append(counter / total_counter)

    #метод для предсказания 
    def predict(self, x):
        P = self.predict_proba(x)
        max_i = 0
        for i in range(1, self.categories_num):
            if P[max_i] < P[i]:
                max_i = i

        return self.categories[max_i]

    #метод для предсказания вероятности
    def predict_proba(self, x):
        x = self.discretize_data([x])[0]

        P = []
        for cat_id in range(self.categories_num):
            p = self.PY[cat_id]
            for i in range(self.variables_num):
                p *= self.PXY[cat_id][i][x[i]]
            P.append(p)

        s = sum(P)
        if s > 0:
            for i in range(len(P)):
                P[i] *= 1 / s

        return P

    #метод для дискретизации данных
    def discretize_data(self, X):
        X2 = []
        for x in X:
            x2 = []
            for i in range(self.variables_num):
                j = 0
                while j < self.intervals_num and x[i] > self.intervals[i][j][1]:
                    j += 1

                if j == self.intervals_num:
                    j -= 1

                x2.append(j)
            X2.append(x2)

        return X2

    #метод для установки дискретного интервала 
    def set_discrete_intervals(self, X):
        self.intervals = []
        for var_num in range(self.variables_num):

            all_values = [x[var_num] for x in X]
            min_value = min(all_values)
            max_value = max(all_values)

            value_span = max_value - min_value
            interval_length = value_span / self.intervals_num

            var_intervals = []
            for i in range(self.intervals_num):
                lower = min_value + i * interval_length
                upper = min_value + (i + 1) * interval_length
                var_intervals.append((lower, upper))

            self.intervals.append(var_intervals)

In [28]:
#функция для тестирования классификатора
def test_classificator(classifier, X, Y):

    correct = 0
    for i in range(len(X)):
        x = X[i]
        y = Y[i]

        result = classifier.predict(x)
        if y == result:
            correct += 1

    return correct

In [29]:
#проверяем модель

#путь до датасета
DATA_FILENAME = "/content/sample_data/wine.csv"
#кол-во данных для тестирования
TRAINING_SET_SIZE = 100

#считываем данные из файла
data = numpy.genfromtxt(DATA_FILENAME, delimiter=',')

In [30]:
#пример строки в датасете
data[0]

array([1.000e+00, 1.423e+01, 1.710e+00, 2.430e+00, 1.560e+01, 1.270e+02,
       2.800e+00, 3.060e+00, 2.800e-01, 2.290e+00, 5.640e+00, 1.040e+00,
       3.920e+00, 1.065e+03])

In [36]:
#разделяем датачет на таргет и фичи
data_Y, data_X = numpy.hsplit(data, [1])

#разделяем датасет на данные для тренеровки и тестирования
train_Y, test_Y, train_X, test_X = train_test_split(
    data_Y, data_X, test_size=TRAINING_SET_SIZE)

#создаем экземпляр классификатора и тренируем его
nbc = BayesClassifier(intervals_num=10, useLaPlace=False)
nbc.fit(train_Y, train_X)

#тестируем классификатор
correct = test_classificator(nbc, test_X, test_Y)

print("кол-во предсказаний:", len(test_X))
print("кол-во правильных предсказаний:", correct, end='')
print(" (%.2f" % (100 * correct / len(test_X)), end='')
print("%)")

кол-во предсказаний: 100
кол-во правильных предсказаний: 75 (75.00%)
